К сожалению, у меня не хватит времени прикрутить сюда распределенную обработку, поэтому в работе использую только часть данных.

In [1]:
import nltk
import re
import torch
import pymorphy2 as pm2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from gensim.models import Word2Vec, FastText
from sklearn.base import TransformerMixin
from sklearn.pipeline import make_pipeline
from joblib import parallel_backend, Parallel, delayed
from tqdm import tqdm
from ast import literal_eval
from sklearn.metrics import pairwise_distances, classification_report
from sklearn.model_selection import train_test_split

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/avagadro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/avagadro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

__load data__

In [10]:
news = pd.read_csv('data/lenta-ru-news.csv', dtype={'topic': str})
news.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [9]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800975 entries, 0 to 800974
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     800975 non-null  object
 1   title   800975 non-null  object
 2   text    800970 non-null  object
 3   topic   738973 non-null  object
 4   tags    773756 non-null  object
 5   date    800975 non-null  object
dtypes: object(6)
memory usage: 36.7+ MB


__preprocessing__

Обрабатывать обучающие данные проще пайплайном, но для единичных пользовательских запросов пайплайн в реализованном виде не очень удобен,
т.к. требует конвертации в pd.DataFrame.

Оптимальное решение пока не нашел, и чтобы не писать тут одни и те же предобработки, пользуюсь конвертацией единичного запроса. В проде такое, на мой взгляд, недопустимо.

In [2]:
class BasicTransformer(TransformerMixin):
    def __init__(self, fields, **kwargs):
        self.fields = fields if isinstance(fields, list) else [fields]
        self.backend = kwargs.pop('backend', 'loky')
        self.n_jobs = kwargs.pop('n_jobs', -1)
        for key, value in kwargs.items():
            setattr(self, key, value)

    def fit(self, X, y=None, **fit_params):
        if 'all' in self.fields:
            self.fields = X.columns
        self.fields = [col for col in self.fields if col in X.columns]
        return self

    def transform(self, X, y=None):
        X = X.copy()
        for f in self.fields:
            X[f] = self.transform_column(X[f])
        return X
    
    def transform_column(self, column):
        return column

In [3]:
class Lowercase(BasicTransformer):
    def transform_column(self, column):
        print(f'Lowercase `{column.name}`')
        return column.str.lower()

class CutRegexp(BasicTransformer):
    def transform_column(self, column):
        print(f'Cut regexp `{column.name}`')
        return column.str.replace(self.pattern, self.fill, regex=True)

class Tokenize(BasicTransformer):
    def transform_column(self, column):
        with parallel_backend(self.backend, n_jobs=self.n_jobs):
            result = Parallel()(delayed(self.tokenizer)(row) for row in tqdm(column.values, desc=f'tokenize `{column.name}`'))
        return result

class ClearStopwords(BasicTransformer):
    def transform_column(self, column):
        def _drop_stopwords(tokens):
            # return [w for w in tokens if w not in self.stopwords]     # NOTE проверить, какой вариант быстрее
            return list(set(tokens).difference(self.stopwords))

        with parallel_backend(self.backend, n_jobs=self.n_jobs):
            result = Parallel()(delayed(_drop_stopwords)(row) for row in tqdm(column.values, desc=f'clearing `{column.name}`'))
        return result
        # return column.apply(lambda tokens: [w for w in tokens if w not in self.stopwords])

class Lemmatize(BasicTransformer):
    morph = pm2.MorphAnalyzer()
    def transform_column(self, column):
        def _lemma(tokens):
            return [self.morph.parse(w)[0].normal_form for w in tokens]

        with parallel_backend(self.backend, n_jobs=self.n_jobs):
            result = Parallel()(delayed(_lemma)(row) for row in tqdm(column.values, desc=f'lemmatize `{column.name}`'))
        return result

In [4]:
# init pipeline
fields = ['title', 'text']
pipeline = make_pipeline(
    Lowercase(fields),
    CutRegexp(fields, pattern=r'\W|\d', fill=' '),      # убрать спецсимволы и цифры
    Tokenize(fields, tokenizer=nltk.tokenize.word_tokenize),
    ClearStopwords(fields, stopwords=nltk.corpus.stopwords.words('russian')),
    Lemmatize(fields)
)

In [5]:
# prepare masks
nans = news['text'].isna() | news['title'].isna()       # drop nan
short = news['text'].str.split().str.len() < 10         # drop short texts - это очень тяжелая операция

topics_mask = news['topic'].value_counts() > 5000       # select data with most frequent topics - это, опять же, для упрощения задачи
topics = news['topic'].value_counts()[topics_mask].index
use_topics = news['topic'].isin(topics)

In [149]:
# preprocess data sample: поскольку времени мало, беру очень маленькую часть данных
data = news[~nans & ~short & use_topics].sample(25000, random_state=23)

prepared = pipeline.fit_transform(data)
prepared.head()

Lowercase `title`
Lowercase `text`
Cut regexp `title`
Cut regexp `text`


lemmatize `text`: 100%|██████████| 25000/25000 [07:29<00:00, 55.56it/s]


,url,title,text,topic,tags,date
69783,https://lenta.ru/news/2003/04/23/teacher/,"[хорватский, библиотека, домашний, учитель, ед...","[сосед, госпитализация, сообщать, ananova, пре...",Из жизни,Все,2003/04/23
69373,https://lenta.ru/news/2003/04/15/syria/,"[буш, готовить, запретить, война, пентагон, си...","[вашингтон, эпизод, принятие, новый, однако, г...",Мир,Все,2003/04/15
722211,https://lenta.ru/news/2018/08/02/latte_for_pre...,"[беременный, средство, вместо, кофе, канадка, ...","[чистить, развесить, заявить, однако, франчайз...",Из жизни,Люди,2018/08/02
240806,https://lenta.ru/news/2008/08/08/mechel1/,"[акция, перенести, размещение, мечел, второй]","[потерять, акция, наметить, доллар, металлурги...",Экономика,Все,2008/08/08
190561,https://lenta.ru/news/2007/05/29/nba/,"[вылет, оказаться, плей, андрей, ют, кириленко...","[сперс, забить, сборная, официальный, чужой, к...",Спорт,Все,2007/05/29


In [151]:
prepared.to_csv('data/prepared.csv')

In [5]:
# load saved
prepared = pd.read_csv('data/prepared.csv', index_col='Unnamed: 0', converters={'title': literal_eval, 'text': literal_eval})

In [6]:
field = 'title'
# field = 'text'
sentences = prepared[field]

# request preprocess
request_text = 'спортивные достижения России'
request = pipeline.fit_transform(pd.DataFrame([request_text], columns=[field]))
request

Lowercase `title`
Cut regexp `title`


lemmatize `title`: 100%|██████████| 1/1 [00:00<00:00, 878.57it/s]


,title
0,"[россия, достижение, спортивный]"


__word2vec__

In [7]:
# word2vec on titles
w2v = Word2Vec(sentences, vector_size=200, window=7, min_count=1)
# build title embeddings
embeddings = sentences.apply(lambda row: np.mean([w2v.wv[word] for word in row if word in w2v.wv], axis=0))
embeddings = np.array(list(embeddings.values))      # recast to np.array
embeddings.shape

(25000, 200)

In [154]:
# NOTE можно сделать отбор или ранжирование кандидатов по топкику

In [8]:
# calc request embedding
request_embedding = request[field].apply(lambda row: np.array([w2v.wv[word] for word in row if word in w2v.wv]).mean(axis=0))
request_embedding = np.array(list(request_embedding.values))      # recast to np.array
request_embedding.shape

(1, 200)

In [11]:
# search nearest
distances = pairwise_distances(request_embedding, embeddings, metric='cosine').flatten()
distances = pd.Series(distances, index=prepared.index)      # restore index

sorted_distances = distances.sort_values(ascending=False)
# top 10 news
top = sorted_distances[:10].index
# overview
news.loc[top]

,url,title,text,topic,tags,date
8463,https://lenta.ru/news/2000/06/02/hockey/,Могильный забивает и проигрывает,Во втором матче финальной серии розыгрыша Кубк...,Спорт,Все,2000/06/02
180927,https://lenta.ru/news/2007/02/21/congress/,В Таллин съезжаются каббалисты,22 января в Таллине откроется Европейский конг...,Бывший СССР,Все,2007/02/21
288524,https://lenta.ru/news/2009/06/26/needlework/,Сикстинскую капеллу вышили крестиком,"Джоанна Лопяновски-Робертс, живущая в США урож...",Культура,Все,2009/06/26
452612,https://lenta.ru/news/2013/01/30/tezzz/,Tequilajazzz воссоединится для перезаписи «Цел...,Группа Tequilajazzz перезапишет альбом «Целлул...,Культура,Музыка,2013/01/30
491914,https://lenta.ru/news/2013/12/24/fbreader/,YotaPhone подружили с читалкой FBReader,Приложение для чтения электронных книг FBReade...,Наука и техника,Гаджеты,2013/12/24
610286,https://lenta.ru/news/2016/08/02/mamont/,Мамонтов добила жажда,Последняя популяция мамонтов на Земле вымерла ...,Наука и техника,Наука,2016/08/02
711456,https://lenta.ru/news/2018/05/03/poslemayskih/,Орангутан загрустил в неволе и растолстел,Орангутан из Бангкокского зоопарка в Таиланде ...,Из жизни,Звери,2018/05/03
715295,https://lenta.ru/news/2018/06/05/newticket/,К брюкам пришили пошлый карман,Японская компания GU выбрала новое местоположе...,Ценности,Стиль,2018/06/05
733934,https://lenta.ru/news/2018/11/06/ronaldinho/,Роналдиньо подозрительно обеднел,Бывший полузащитник «Барселоны» и сборной Браз...,Спорт,Футбол,2018/11/06
735243,https://lenta.ru/news/2018/11/16/petrosyan/,Петросян и Степаненко развелись,Брак юмористов Евгения Петросяна и Елены Степа...,Интернет и СМИ,ТВ и радио,2018/11/16


__FastText__

In [13]:
ft = FastText(sentences, vector_size=500)
# build title embeddings
embeddings = sentences.apply(lambda row: np.mean([ft.wv[word] for word in row if word in ft.wv], axis=0))
embeddings = np.array(embeddings.tolist())      # recast to np.array
embeddings.shape

(25000, 500)

In [158]:
# NOTE можно сделать отбор или ранжирование кандидатов по топкику

In [14]:
# calc request embedding
request_embedding = request[field].apply(lambda row: np.array([ft.wv[word] for word in row if word in ft.wv]).mean(axis=0))
request_embedding = np.array(request_embedding.tolist())      # recast to np.array
request_embedding.shape

(1, 500)

In [15]:
# search nearest
distances = pairwise_distances(request_embedding, embeddings, metric='cosine').flatten()
distances = pd.Series(distances, index=prepared.index)      # restore index

sorted_distances = distances.sort_values(ascending=False)
# top 10 news
top = sorted_distances[:10].index
# overview
news.loc[top]

,url,title,text,topic,tags,date
31685,https://lenta.ru/news/2001/07/26/cbr/,Центробанк накопил 36 миллиардов долларов,Золотовалютные резервы России за период с 13 п...,Экономика,Все,2001/07/26
130166,https://lenta.ru/news/2005/09/13/sell/,"Ford Motor продает Hertz за 5,6 миллиарда долл...",Концерн Ford Motor продает свое подразделение ...,Экономика,Все,2005/09/13
138667,https://lenta.ru/news/2005/12/16/invest/,Чубайс ликвидирует энергодефицит за 12 миллиар...,"Глава РАО ""ЕЭС России"" Анатолий Чубайс намерен...",Экономика,Все,2005/12/16
160043,https://lenta.ru/news/2006/07/26/tehnosila/,"""Техносила"" построит себе склад за 100 миллион...","Торговая группа ""Техносила"" построит собственн...",Дом,Все,2006/07/26
167613,https://lenta.ru/news/2006/10/09/symantec/,Symantec заработает на антивирусах 10 миллиард...,Главный исполнительный директор Symantec Джон ...,Интернет и СМИ,Все,2006/10/09
246340,https://lenta.ru/news/2008/09/17/samsung/,Samsung предложил за SanDisk почти шесть милли...,Корейская корпорация Samsung Electronics предл...,Наука и техника,Все,2008/09/17
248000,https://lenta.ru/news/2008/09/29/digg/,Digg оценили в 175 миллионов долларов,В ходе последнего раунда финансирования социал...,Интернет и СМИ,Все,2008/09/29
443716,https://lenta.ru/news/2012/11/20/myspace/,На перезапуск MySpace попросят 50 миллионов до...,"Холдинг Interactive Media, владеющий музыкальн...",Интернет и СМИ,Все,2012/11/20
590175,https://lenta.ru/news/2016/04/07/kubanbillion/,«Кубань» задолжала более миллиарда рублей,"Долги краснодарского клуба «Кубань», выступающ...",Спорт,Футбол,2016/04/07
637519,https://lenta.ru/news/2016/12/26/veb/,ВЭБ предсказал себе 130 миллиардов рублей убытков,Внешэкономбанк по итогам года может получить у...,Экономика,Госэкономика,2016/12/26


__why it doesn't work__

In [161]:
w2v.wv.most_similar(request[field][0], topn=10)

[('второй', 0.9986008405685425),
 ('место', 0.9985744953155518),
 ('объяснить', 0.9985387921333313),
 ('рассказать', 0.9985325336456299),
 ('последний', 0.9985318183898926),
 ('матч', 0.9985306262969971),
 ('германия', 0.9985304474830627),
 ('реклама', 0.9985286593437195),
 ('оон', 0.9985216856002808),
 ('украинский', 0.9985213279724121)]

In [162]:
ft.wv.most_similar(request[field][0], topn=10)

[('агрессия', 0.9999939799308777),
 ('розовый', 0.9999921917915344),
 ('официальный', 0.9999920725822449),
 ('криминальный', 0.9999911189079285),
 ('белоруссия', 0.9999909400939941),
 ('россельхознадзор', 0.9999908804893494),
 ('роналдый', 0.9999907612800598),
 ('региональный', 0.9999905824661255),
 ('валютный', 0.9999904036521912),
 ('дешёвый', 0.9999901056289673)]

Наипростейший вариант поиска ближайших эмбеддингов - по косинусному расстоянию - в данном случае работает неэффективно.

__пример классификатора топика__

In [12]:
# кластеризатор по топикам
idx2topic = dict(enumerate(prepared['topic'].unique()))
topic2idx = {v: k for k,v in idx2topic.items()}

true_labels = prepared['topic'].map(topic2idx)

# train/valid split
embs = pd.DataFrame(embeddings, index=true_labels.index)    # restore embedding indices
train, valid = train_test_split(embs.index, test_size=0.2, stratify=true_labels, random_state=19)

In [19]:
from sklearn.mixture import BayesianGaussianMixture, GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from lightgbm import LGBMClassifier

# fit classifier
# model = BayesianGaussianMixture(n_components=len(topic2idx), n_init=1, random_state=17)
# model = KNeighborsClassifier(n_neighbors=len(topic2idx), n_jobs=-1)
# model = KMeans(n_clusters=len(topic2idx), random_state=17)
model = LGBMClassifier(n_estimators=500, learning_rate=0.1, max_depth=5, num_leaves=31, n_jobs=-1, random_state=17)
model.fit(embs.loc[train], true_labels[train])
pred_labels = model.predict(embs.loc[valid])
# print report
report = classification_report(true_labels[valid], pred_labels)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       198
           1       0.33      0.45      0.38       905
           2       0.37      0.46      0.41       553
           3       0.61      0.49      0.54       431
           4       0.27      0.23      0.25       357
           5       0.00      0.00      0.00       139
           6       0.34      0.56      0.42      1086
           7       0.13      0.06      0.08       315
           8       0.19      0.14      0.16       361
           9       0.29      0.13      0.18       359
          10       0.00      0.00      0.00        53
          11       0.50      0.03      0.05        39
          12       0.47      0.19      0.27       151
          13       0.00      0.00      0.00        53

    accuracy                           0.34      5000
   macro avg       0.25      0.19      0.20      5000
weighted avg       0.31      0.34      0.31      5000



/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [71]:
class EmbeddingsDataset():
    def __init__(self, X, y):
        self.X = X.reset_index(drop=True)
        self.y = y.reset_index(drop=True)

    def __getitem__(self, index):
        return self.X.loc[index].values, self.y.loc[index].values
    
    def __len__(self):
        return self.X.shape[0]


class Cell(torch.nn.Module):
    def __init__(self, inp, out, *, act=torch.relu, drop=0):
        super().__init__()
        self.linear = torch.nn.Linear(inp, out)
        self.activation = act
        self.bn = torch.nn.BatchNorm1d(out)
        self.dp = torch.nn.Dropout(drop) if drop else None
    
    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        x = self.bn(x)
        if self.dp is not None:
            x = self.dp(x)
        return x


class Net(torch.nn.Module):
    def __init__(self, inp, out):
        super().__init__()
        self.cell1 = Cell(inp, 512, act=torch.relu, drop=0.1)
        self.cell2 = Cell(512, 256, act=torch.relu, drop=0.1)
        self.cell3 = Cell(256, out, act=torch.tanh, drop=0.1)
        
    def forward(self, x):
        x = self.cell1(x)
        x = self.cell2(x)
        x = self.cell3(x)
        return torch.softmax(x, dim=0)

In [72]:
EPOCHS = 5

# init network
device = 'cuda'
net = Net(200, len(topic2idx)).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

# make data loaders
onehot_labels = pd.DataFrame(np.eye(true_labels.max() + 1)[true_labels.values], index=true_labels.index)
emb_train_dataset = EmbeddingsDataset(embs.loc[train], onehot_labels.loc[train])
emb_valid_dataset = EmbeddingsDataset(embs.loc[valid], onehot_labels.loc[train])
train_loader = torch.utils.data.DataLoader(emb_train_dataset, batch_size=64, shuffle=True)
valid_loader = torch.utils.data.DataLoader(emb_valid_dataset, batch_size=64, shuffle=True)

In [73]:
# train
net.train()
for ep in range(EPOCHS):
    sum_loss, items = 0.0, 0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Epoch {ep + 1}/{EPOCHS}')
    for i, batch in pbar:
        inputs, labels = batch[0].to(device).float(), batch[1].to(device).float()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()
        items += len(labels)
        pbar.set_postfix({'cumulative loss per item': sum_loss / items})
print('\nDone.')

Epoch 5/5: 100%|██████████| 313/313 [00:04<00:00, 69.06it/s, cumulative loss per item=0.00469]


Done.


In [74]:
# evaluation
net.eval()
pred_labels = net(torch.as_tensor(embs.loc[valid].values, device=device)).detach().cpu().argmax(axis=1)
report = classification_report(true_labels[valid], pred_labels)
print(report)

              precision    recall  f1-score   support

           0       0.06      0.42      0.11       198
           1       0.26      0.11      0.16       905
           2       0.32      0.44      0.37       553
           3       0.29      0.65      0.40       431
           4       0.00      0.00      0.00       357
           5       0.03      0.07      0.05       139
           6       0.34      0.02      0.04      1086
           7       0.14      0.00      0.01       315
           8       0.00      0.00      0.00       361
           9       0.13      0.19      0.16       359
          10       0.00      0.00      0.00        53
          11       0.01      0.08      0.02        39
          12       0.04      0.06      0.05       151
          13       0.00      0.00      0.00        53

    accuracy                           0.17      5000
   macro avg       0.12      0.15      0.10      5000
weighted avg       0.20      0.17      0.13      5000



/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/avagadro/anaconda3/envs/jupyter_default/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [163]:
#